In [0]:
spark.sql("""
          CREATE TABLE IF NOT EXISTS dq_table_metrics(
              table_name STRING,
              run_date DATE,
              row_count LONG,
              null_stats MAP<STRING, LONG>,
              distinct_stats MAP<STRING, LONG>,
              schema_hash STRING,
              active_columns ARRAY<STRING>,
              collected_ts TIMESTAMP
          )
          USING DELTA
          """)

In [0]:
import hashlib
from pyspark.sql import functions as F

def profile_table(table_name, run_date):
    df = spark.table(table_name).filter(F.col("order_date") == run_date)

    row_count = df.count()

    null_stats = {
        c: df.filter(F.col(c).isNull()).count()
        for c in df.columns
    }

    distinct_stats = {
        c: df.select(c).distinct().count()
        for c in df.columns
    }

    schema_string = "|".join(
        [f"{f.name}:{f.dataType.simpleString()}" for f in df.schema.fields]
    )

    schema_hash = hashlib.md5(schema_string.encode()).hexdigest()

    active_columns = [
        c for c in df.columns
        if df.filter(F.col(c).isNotNull()).limit(1).count() > 0
    ]

    metrics_df = spark.createDataFrame([(
        table_name,
        run_date,
        row_count,
        null_stats,
        distinct_stats,
        schema_hash,
        active_columns
    )],[
        "table_name",
        "run_date",
        "row_count",
        "null_stats",
        "distinct_stats",
        "schema_hash",
        "active_columns"
    ]).withColumn("collected_ts", F.current_timestamp())

    metrics_df.write.mode("append").saveAsTable("dq_table_metrics")

In [0]:
# spark.sql("""
#           ALTER TABLE dq_table_metrics ADD COLUMN (
#               active_columns ARRAY<STRING>
#           )
#           """)

In [0]:
dates = [
    r.order_date for r in 
    spark.sql("SELECT DISTINCT order_date FROM retail_orders").collect()
]

for d in dates:
    profile_table("retail_orders", d)


In [0]:
x  = spark.sql("""
        SELECT
            run_date,
            row_count,
            null_stats["customer_id"],
            distinct_stats["product_id"] as product_cardinality,
            -- schemahash,
            active_columns
        FROM
            dq_table_metrics 
        ORDER BY
            run_date""")
        #   """).show(truncate=False)

In [0]:
display(x)